In [ ]:
import pandas as pd
import numpy as np
import calendar
from google.colab import drive
import ipywidgets as widgets
from IPython.display import display, clear_output

pd.set_option('display.max_columns', None)
pd.set_option('styler.render.max_rows', None)
pd.set_option('display.float_format', '{:,.1f}'.format)

In [ ]:
# ===============================================================
# 2) Mount Drive & Load Files
# ===============================================================

drive.mount('/content/drive')

# Paths
file_path1 = "/content/drive/MyDrive/IOE 424 (Gemline)/Data/SKU Data/SKU Attributes Report V2.xlsx"
file_path2 = "/content/drive/MyDrive/IOE 424 (Gemline)/Data/Sales and Pricing data/V3_Jan. 2022 to August 2025 Sales and Price data.xlsx"

# Load SKU attributes
df_sku_attributes = pd.read_excel(file_path1, engine="openpyxl")
df_sku_attributes.columns = df_sku_attributes.columns.str.strip()
df_sku_attributes["Item"] = df_sku_attributes["Item"].astype(str).str.strip()

print(f"SKU attributes loaded: {len(df_sku_attributes):,} rows")


Mounted at /content/drive
SKU attributes loaded: 2,409 rows


In [ ]:
# ===============================================================
# 3) Define Category Mapping & Column Resolver
# ===============================================================

category_dict = {
    "*":      "COMMON",
    "BACKP":  "Backpack",
    "BBAGS":  "Business Bag",
    "COOL":   "Cooler",
    "DRINK":  "Drinkware",
    "DUF":    "Duffels",
    "FOOD":   "Food",
    "PAD":    "Padfolio",
    "STNRY":  "Stationery",
    "TECH":   "Technology",
    "TOTE":   "Totes",
    "TRAVL":  "Travel",
    "WRITE":  "Writing",
    "LFSTY":  "Lifestyle",
    "PPE":    "Personal Protective Equip.",
    "UMBR":   "Umbrellas",
    "HDWR":   "Headwear",
    "PKG":    "Packaging",
    "BLKTW":  "Blankets and Towels",
    "HOME":   "Home",
    "OFFCE":  "Office",
    "OUTDR":  "Outdoor",
    "SLWPK":  "Slings and Waist Packs",
}

def find_col(cols, target):
    """Find matching column name in a case-insensitive way."""
    t = target.lower()
    for c in cols:
        if c.lower() == t:
            return c
    def norm(s): return s.lower().replace(' ', '').replace('_', '')
    nt = norm(target)
    for c in cols:
        if norm(c) == nt:
            return c
    return None

col_category = find_col(df_sku_attributes.columns, "Category")
col_intro = find_col(df_sku_attributes.columns, "Intro Code")

if col_category is None:
    raise KeyError("Could not find 'Category' column in attributes file.")


In [ ]:
# ===============================================================
# 4) Clean & Prepare SKU Attributes
# ===============================================================

df_attr_clean = df_sku_attributes.copy()

df_attr_clean["Item"] = df_attr_clean["Item"].astype(str).str.strip()
df_attr_clean[col_category] = df_attr_clean[col_category].astype(str).str.strip()

na_like = {"", "na", "n/a", "null", "none", "nan", "-", "--", "—"}
df_attr_clean[col_category] = df_attr_clean[col_category].apply(
    lambda x: None if str(x).strip().lower() in na_like else x
)
df_attr_clean = df_attr_clean.dropna(subset=[col_category])

print(f"Clean attribute rows: {len(df_attr_clean):,}")


Clean attribute rows: 2,409


In [ ]:
# ===============================================================
# 5) Load & Combine Sales Data (2022–2025)
# ===============================================================

sheet_names = [str(y) for y in range(2022, 2026)]
sheet_dict = pd.read_excel(file_path2, sheet_name=sheet_names, engine="openpyxl")

rename_map = {"Planned Ship Date/Time": "ShipDate", "Sales Qty": "SalesQty"}
frames = []

for year, df in sheet_dict.items():
    df = df.copy()
    df.columns = df.columns.str.strip()
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})
    if "ShipDate" in df.columns:
        df["ShipDate"] = pd.to_datetime(df["ShipDate"], errors="coerce")
    if "SalesQty" in df.columns:
        df["SalesQty"] = pd.to_numeric(df["SalesQty"], errors="coerce").fillna(0)
    if "Item" in df.columns:
        df["Item"] = df["Item"].astype(str).str.strip()
    df["year"] = int(year)
    frames.append(df)

combined_df = pd.concat(frames, ignore_index=True).dropna(subset=["ShipDate"]).reset_index(drop=True)
print(f"Combined sales rows: {len(combined_df):,}")


Combined sales rows: 624,603


In [ ]:
# ===============================================================
# 6) Merge Category into Sales & Clean
# ===============================================================

sku_map = (
    df_attr_clean[["Item", col_category]]
      .drop_duplicates(subset="Item")
      .rename(columns={col_category: "Category"})
      .assign(Category=lambda d: d["Category"].astype("string").str.strip())
      .set_index("Item")["Category"]
)

combined_df["Category"] = combined_df["Item"].map(sku_map)
combined_df["CategoryName"] = combined_df["Category"].map(category_dict)

# Drop rows without category or bad categories
mask_badcat = combined_df["Category"].astype(str).str.upper().isin(["PPE", "PAD"])
mask_na = combined_df["Category"].isna()
combined_df = combined_df[~(mask_badcat | mask_na)].reset_index(drop=True)

print(f"Remaining rows after filtering: {len(combined_df):,}")


Remaining rows after filtering: 419,410


In [ ]:
# ===============================================================
# Peak Month Analysis — Category Level (NO FILTERS; show all)
# (with 33% confidence augmentation of Peak_Month text)
# ===============================================================

import pandas as pd, numpy as np, calendar

YEARS_USE = {2022, 2023, 2024}
G = ['CategoryName']

def prep_sales(df):
    df = df.copy()
    df['ShipDate']  = pd.to_datetime(df['ShipDate'], errors='coerce')
    df             = df.dropna(subset=['ShipDate'])
    df['year']     = df['ShipDate'].dt.year
    df['month_num']= df['ShipDate'].dt.month
    return df[df['year'].isin(YEARS_USE)]

def monthly_sales(df):
    return (
        df.groupby(G + ['year', 'month_num'], dropna=False)['SalesQty']
          .sum()
          .reset_index(name='Monthly_Sales')
    )

def wrap_prev(m, k=1): return ((int(m) - 1 - k) % 12) + 1
def wrap_next(m, k=1): return ((int(m) - 1 + k) % 12) + 1

# --- Build base monthly dataset (no "full year" filter)
monthly = monthly_sales(prep_sales(combined_df))

# --- Identify per-year peak month for each category
monthly['rank_in_year'] = monthly.groupby(G + ['year'])['Monthly_Sales'] \
                                 .rank(method='first', ascending=False)
top1 = monthly.loc[monthly['rank_in_year'] == 1,
                   G + ['year', 'month_num', 'Monthly_Sales']]

# --- How many years present (any data) per category
n_years_present = (monthly.groupby(G, as_index=False)['year']
                          .nunique()
                          .rename(columns={'year':'N_years_present'}))

# --- Peak month mode + confidence across the available years
mode_top = (top1.groupby(G, as_index=False)['month_num']
                 .agg(lambda s: s.mode().iloc[0] if len(s) else np.nan)
                 .rename(columns={'month_num':'Peak_MonthNum'}))

yrs_top = (top1.groupby(G, as_index=False)['year']
                .nunique().rename(columns={'year':'N_years_with_top'}))

hits = (top1.groupby(G + ['month_num'], as_index=False)
             .size().rename(columns={'size':'Hits'}))

conf = (mode_top
        .merge(hits, left_on=G + ['Peak_MonthNum'], right_on=G + ['month_num'], how='left')
        .drop(columns=['month_num'])
        .merge(yrs_top, on=G, how='left'))
conf['Hits'] = conf['Hits'].fillna(0)
conf['Peak_Confidence'] = np.where(conf['N_years_with_top']>0,
                                   conf['Hits']/conf['N_years_with_top'],
                                   np.nan)

# --- Averages (over whatever months exist)
avg_month_sales = (monthly.groupby(G + ['month_num'], as_index=False)['Monthly_Sales']
                          .mean().rename(columns={'Monthly_Sales':'Avg_Monthly_Sales'}))

avg_peak_sales = (top1.groupby(G, as_index=False)['Monthly_Sales']
                       .mean().rename(columns={'Monthly_Sales':'Avg Peak Month Sales'}))

avg_all_months = (monthly.groupby(G, as_index=False)['Monthly_Sales']
                        .mean().rename(columns={'Monthly_Sales':'Avg_AllMonths_Sales'}))

# --- Merge everything (NO FILTERS)
plan_all = (mode_top
            .merge(conf[G + ['Peak_Confidence']], on=G, how='left')
            .merge(n_years_present, on=G, how='left')
            .merge(avg_peak_sales, on=G, how='left')
            .merge(avg_all_months, on=G, how='left'))

# ===============================================================
# Augment Peak_Month text when confidence is ~33% (1 of 3 years)
# ===============================================================

# All distinct peak months per Category across years
peak_months_map = (
    top1.groupby('CategoryName')['month_num']
        .apply(lambda s: sorted(s.unique()))
        .to_dict()
)

# Identify categories where the chosen mode has Hits==1 out of N_years_with_top==3
conf_33 = (
    conf.assign(_is33 = (conf['N_years_with_top'].eq(3) & conf['Hits'].eq(1)))
        .set_index('CategoryName')['_is33']
        .to_dict()
)

# Ensure Peak_MonthNum is present (it is from mode_top merge above)
def augment_peak_month(row):
    if pd.isna(row['Peak_MonthNum']):
        return np.nan
    cat = row['CategoryName']
    primary_num = int(row['Peak_MonthNum'])
    primary_abbr = calendar.month_abbr[primary_num]

    # Only augment when confidence is 33% (1 of 3)
    if conf_33.get(cat, False):
        all_nums = peak_months_map.get(cat, [])
        all_abbrs = [calendar.month_abbr[int(m)] for m in all_nums]
        # keep order by month number; list others excluding the primary
        others = [m for m in all_abbrs if m != primary_abbr]
        if others:
            return f"{primary_abbr} + {' + '.join(others)}"
    return primary_abbr

# Temporary text column; will assign to Peak_Month below
plan_all['Peak_Month_text'] = plan_all.apply(augment_peak_month, axis=1)

# --- Adjacent month averages (relative to Peak_MonthNum)
def get_adjacent_sales(row):
    if pd.isna(row['Peak_MonthNum']):
        return pd.Series([np.nan, np.nan, np.nan],
                         index=['Avg Peak Month (-2) Sales',
                                'Avg Peak Month (-1) Sales',
                                'Avg Peak Month (+1) Sales'])
    m  = int(row['Peak_MonthNum'])
    m2, m1, mp = wrap_prev(m, 2), wrap_prev(m, 1), wrap_next(m, 1)
    mask = (avg_month_sales['CategoryName'] == row['CategoryName'])
    def val(mm):
        s = avg_month_sales.loc[mask & (avg_month_sales['month_num']==mm),'Avg_Monthly_Sales']
        return s.squeeze() if not s.empty else np.nan
    return pd.Series([val(m2), val(m1), val(mp)],
                     index=['Avg Peak Month (-2) Sales',
                            'Avg Peak Month (-1) Sales',
                            'Avg Peak Month (+1) Sales'])

plan_all = pd.concat([plan_all, plan_all.apply(get_adjacent_sales, axis=1)], axis=1)

# --- Readable columns
plan_all['Peak_Month']        = plan_all['Peak_Month_text']
plan_all['Peak_Confidence_%'] = (plan_all['Peak_Confidence']*100).round(1)
plan_all['Avg Peak Month Sales']  = plan_all['Avg Peak Month Sales'].round(1)
plan_all['Avg_AllMonths_Sales']   = plan_all['Avg_AllMonths_Sales'].round(1)
plan_all['Push_MonthNum']     = plan_all['Peak_MonthNum'].apply(lambda m: ((int(m)-2)%12)+1 if pd.notna(m) else np.nan)
plan_all['Push_Month']        = plan_all['Push_MonthNum'].map(lambda m: calendar.month_abbr[int(m)] if pd.notna(m) else np.nan)

# --- Final table (ALL categories)
cols_all = [
    'CategoryName',
    'N_years_present',
    'Peak_Month',
    'Peak_Confidence_%',
    'Avg Peak Month (-2) Sales',
    'Avg Peak Month (-1) Sales',
    'Avg Peak Month Sales',
    'Avg Peak Month (+1) Sales',
    'Avg_AllMonths_Sales',
    'Push_Month'
]
peak_summary_all = plan_all[cols_all].sort_values('CategoryName').reset_index(drop=True)

# --- Style + display (shows ALL rows)
def bold_peak(s):
    return ['font-weight:700;' if not pd.isna(v) else '' for v in s]

styled_all = (
    peak_summary_all.style
      .set_table_styles([
          {'selector':'th, td','props':[('border','1px solid #5A9BD5')]},
          {'selector':'table','props':[('border-collapse','collapse')]},
          {'selector':'th','props':[('text-align','center')]},
      ])
      .set_properties(**{'text-align':'center'})
      .apply(bold_peak, subset=['Avg Peak Month Sales'])
      .background_gradient(subset=['Peak_Confidence_%'], cmap='RdYlGn', vmin=0, vmax=100)
      .format({
          'Peak_Confidence_%': '{:.1f}%',
          'Avg Peak Month (-2) Sales': '{:,.1f}',
          'Avg Peak Month (-1) Sales': '{:,.1f}',
          'Avg Peak Month Sales': '{:,.1f}',
          'Avg Peak Month (+1) Sales': '{:,.1f}',
          'Avg_AllMonths_Sales': '{:,.1f}',
      })
)

display(styled_all)


,CategoryName,N_years_present,Peak_Month,Peak_Confidence_%,Avg Peak Month (-2) Sales,Avg Peak Month (-1) Sales,Avg Peak Month Sales,Avg Peak Month (+1) Sales,Avg_AllMonths_Sales,Push_Month
0,Backpack,3,Apr + May + Aug,33.3%,"27,576.0","36,738.7","52,136.3","50,961.3","38,431.5",Mar
1,Blankets and Towels,3,Jul + Oct + Nov,33.3%,"1,640.0","2,268.0","10,149.7","2,566.3","3,292.8",Jun
2,Business Bag,3,Aug + Sep + Nov,33.3%,"3,487.7","3,935.0","5,134.0","4,220.0","3,482.8",Jul
3,Cooler,3,Aug + Nov + Dec,33.3%,"48,315.3","49,464.0","72,024.7","59,366.0","52,462.2",Jul
4,Drinkware,3,Dec,100.0%,"66,694.3","67,425.7","94,005.7","69,301.7","64,545.7",Nov
5,Duffels,3,Feb + Apr + Sep,33.3%,"17,427.3","10,410.7","24,645.3","17,282.3","15,471.0",Jan
6,Food,3,Apr + Nov + Dec,33.3%,"2,936.7","2,577.0","12,412.0","1,546.0","3,713.2",Mar
7,Headwear,1,Sep,100.0%,396.0,"4,845.0","6,577.0","5,436.0","4,364.0",Aug
8,Home,3,Nov,66.7%,"17,396.3","23,967.3","30,175.7","23,205.0","16,678.6",Oct
9,Office,3,Jun + Aug + Oct,33.3%,"16,558.3","20,641.7","28,510.0","18,116.7","20,181.1",May


In [ ]:
# create the documentation for how we did this